In [61]:
class Graph(object):
    
    def __init__(self, graph_dict=None):
        """ initializes a graph object 
            If no dictionary or None is given, 
            an empty dictionary will be used
        """
        if graph_dict == None:
            graph_dict = {}
            pass
        self.__graph_dict = graph_dict
        self.__path_occupied = []
        pass
    
    def vertices(self):
        """ returns the vertices of a graph """
        return list(self.__graph_dict.keys())
    
    def edges(self):
        """ returns the edges of a graph """
        return self.__generate_edges()
    
    def add_vertex(self, vertex):
        """ If the vertex "vertex" is not in 
            self.__graph_dict, a key "vertex" with an empty
            list as a value is added to the dictionary. 
            Otherwise nothing has to be done. 
        """
        if vertex not in self.__graph_dict:
            self.__graph_dict[vertex] = []
            pass
        pass
    
    def add_edge(self, edge):
        """ assumes that edge is of type set, tuple or list; 
            between two vertices can be multiple edges! 
        """
        edge = set(edge)
        (vertex1, vertex2) = tuple(edge)
        if vertex1 in self.__graph_dict:
            self.__graph_dict[vertex1].append(vertex2)
            pass
        else:
            self.__graph_dict[vertex1] = [vertex2]
            pass
        pass
    
    def __generate_edges(self):
        """ A static method generating the edges of the 
            graph "graph". Edges are represented as sets 
            with one (a loop back to the vertex) or two 
            vertices 
        """
        edges = []
        for vertex in self.__graph_dict:
            for neighbour in self.__graph_dict[vertex]:
                if {neighbour, vertex} not in edges:
                    edges.append({vertex, neighbour})
                    pass
                pass
            pass
        return edges
    
    def __str__(self):
        res = "vertices: "
        for k in self.__graph_dict:
            res += str(k) + " "
            pass
        res += "\nedges: "
        for edge in self.__generate_edges():
            res += str(edge) + " "
            pass
        return res
    
    def find_path(self, start_vertex, end_vertex, path=None):
        """ find a path from start_vertex to end_vertex 
            in graph """
        if path == None:
            path = []
            pass
        graph = self.__graph_dict
        path = path + [start_vertex]
        if start_vertex == end_vertex:
            return path
        if start_vertex not in graph:
            return None
        for vertex in graph[start_vertex]:
            if vertex not in path:
                extended_path = self.find_path(vertex, end_vertex, path)
                if extended_path:
                    return extended_path
                pass
            pass
        return None
    
    def find_all_paths(self, start_vertex, end_vertex, path=[]):
        """ find all paths from start_vertex to 
            end_vertex in graph """
        graph = self.__graph_dict
        path = path + [start_vertex]
        if start_vertex == end_vertex:
            return [path]
        if start_vertex not in graph:
            return []
        paths = []
        for vertex in graph[start_vertex]:
            if vertex not in path:
                extended_paths = self.find_all_paths(vertex, end_vertex, path)
                for p in extended_paths:
                    paths.append(p)
                    pass
                pass
            pass
        return paths
    
    def find_onepair_path(self, start_vertex, end_vertex):
        # 找出所有路径
        print('All paths from vertex "'+ start_vertex + '"' + ' to vertex '+ '"' + end_vertex + '"'+ ':')
        paths = self.find_all_paths(start_vertex, end_vertex)
        print("所有路径：")
        print(paths)
        # 找出冲突路径
        tem = []
        for i in paths:
            for j in self.__path_occupied:
                for m in range(len(i) -1):
                    for n in range(len(j) - 1):
                        if i[m] == j[n] and i[m+1] == j[n+1] and i not in tem:
                            tem.append(i)
                            pass
                        pass
                    pass
                pass
            pass
        print("冲突路径：")
        print(tem)
        # 从路径中删去冲突路径
        for ele in tem:
            paths.remove(ele)
            pass
        print("可选路径：")
        print(paths)
        # 在剩下的可选路径中找最短的
        min = 100
        min_path = []
        for ele in paths:
            if len(ele) < min:
                min_path = ele
                min = len(ele)
                pass
            pass
        print("可选路径中的最短路径：")
        print(min_path)
        return min_path
    
    def find_disjoint_path(self, pairs):
        if pairs == []:
            return
        # 找出所有pairs中最短路径
        tem_paths = []
        min = 100
        min_index = 0
        index = 0
        min_path = []
        for pair in pairs:
            path = self.find_onepair_path(pair[0], pair[1])
            if len(path) < min:
                min = len(path)
                min_index = index
                min_path = path
                pass
            index += 1
            pass
        # 从pairs中删去已经找到路径的点对，并将该点对的路径存入self.__path_occupied中
        pairs.pop(min_index)
        self.__path_occupied.append(min_path)
        self.find_disjoint_path(pairs)
        return self.__path_occupied

In [62]:
if __name__ == "__main__":
    g = { "s1": ["s2"],
          "s2": ["t1","t2","t3"],
          "s3": ["s2"],
          "t1": [],
          "t2": [],
          "t3": []
        }
    
    g1 = { "a" : ["d", "f"],
           "b" : ["c"],
           "c" : ["b", "c", "d", "e", "f"],
           "d" : ["a", "c"],
           "e" : ["c"],
           "f" : ["d", "c"]
          }
    
    g2 = { "s1": ["a","c"],
          "s2": ["s1"],
          "s3": ["c"],
          "a" : ["b"],
          "b" : ["t1"],
          "c" : ["t1","t2"],
          "t1": ["t3"],
          "t2": [],
          "t3": []
         }
    
    graph = Graph(g2)
    pairs = [["s1","t1"], ["s2","t2"],["s3","t3"]]

    #graph.find_onepair_path("a", "e")
    paths = graph.find_disjoint_path(pairs)
    print("最终：")
    print(paths)

All paths from vertex "s1" to vertex "t1":
所有路径：
[['s1', 'a', 'b', 't1'], ['s1', 'c', 't1']]
冲突路径：
[]
可选路径：
[['s1', 'a', 'b', 't1'], ['s1', 'c', 't1']]
可选路径中的最短路径：
['s1', 'c', 't1']
All paths from vertex "s2" to vertex "t2":
所有路径：
[['s2', 's1', 'c', 't2']]
冲突路径：
[]
可选路径：
[['s2', 's1', 'c', 't2']]
可选路径中的最短路径：
['s2', 's1', 'c', 't2']
All paths from vertex "s3" to vertex "t3":
所有路径：
[['s3', 'c', 't1', 't3']]
冲突路径：
[]
可选路径：
[['s3', 'c', 't1', 't3']]
可选路径中的最短路径：
['s3', 'c', 't1', 't3']
All paths from vertex "s2" to vertex "t2":
所有路径：
[['s2', 's1', 'c', 't2']]
冲突路径：
[['s2', 's1', 'c', 't2']]
可选路径：
[]
可选路径中的最短路径：
[]
All paths from vertex "s3" to vertex "t3":
所有路径：
[['s3', 'c', 't1', 't3']]
冲突路径：
[['s3', 'c', 't1', 't3']]
可选路径：
[]
可选路径中的最短路径：
[]
All paths from vertex "s3" to vertex "t3":
所有路径：
[['s3', 'c', 't1', 't3']]
冲突路径：
[['s3', 'c', 't1', 't3']]
可选路径：
[]
可选路径中的最短路径：
[]
最终：
[['s1', 'c', 't1'], [], []]


In [ ]:
    print('The path from vertex "a" to vertex "e":')
    path = graph.find_path("a", "e")
    print(path)
    
    path_occupied = []
    # 找出所有路径
    print('All paths from vertex "a" to vertex "b":')
    paths = graph.find_all_paths("a", "b")
    print(paths)
    # 找出冲突路径
    tem = []
    for i in paths:
        for j in path_occupied:
            for m in range(len(i) -1):
                for n in range(len(j) - 1):
                    if i[m] == j[n] and i[m+1] == j[n+1] and i not in tem:
                        tem.append(i)
                        pass
                    pass
                pass
            pass
        pass
    print(tem)
    # 从路径中删去冲突路径
    for ele in tem:
        paths.remove(ele)
        pass
    print(paths)
    # 在剩下的可选路径中找最短的
    min = 100
    min_path = []
    for ele in paths:
        if len(ele) < min:
            min_path = ele
            min = len(ele)
            pass
        pass
    print(min_path)
    path_occupied.append(min_path)
    print(path_occupied)